<a href="https://colab.research.google.com/github/mainabhihoon/simp_learn_repos/blob/main/Car_Rental_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
# No 'time' import is needed here, as we only use datetime

class Car:
    """
    Represents a car available for rent.

    Attributes:
        car_id (int): A unique identifier for the car.
        make (str): The car's manufacturer.
        model (str): The car's model name.
        year (int): The car's manufacturing year.
        is_available (bool): A flag indicating if the car is currently available for rent.
    """
    def __init__(self, car_id, make, model, year):
        self.car_id = car_id
        self.make = make
        self.model = model
        self.year = year
        self.is_available = True

    def __str__(self):
        """
        Provides a string representation of the Car object.
        """
        status = "Available" if self.is_available else "Rented"
        return f"ID: {self.car_id} | {self.year} {self.make} {self.model} | Status: {status}"

class Customer:
    """
    Represents a customer of the car rental service.

    Attributes:
        customer_id (int): A unique identifier for the customer.
        name (str): The customer's full name.
        rented_cars (dict): A dictionary of rented Car objects.
    """
    def __init__(self, customer_id, name):
        self.customer_id = customer_id
        self.name = name
        self.rented_cars = {} # Stores {car_id: Car_object}

    def __str__(self):
        """
        Provides a string representation of the Customer object.
        """
        return f"Customer: {self.name} (ID: {self.customer_id})"

class RentalSystem:
    """
    Manages the car rental process, including inventory and customer data.

    Attributes:
        cars (dict): A dictionary mapping car IDs to Car objects.
        customers (dict): A dictionary mapping customer IDs to Customer objects.
        rental_records (dict): A nested dictionary of rental details.
            Format: {customer_id: {car_id: {'start_time': datetime, 'basis': str}, ...}}
        rental_rates (dict): A dictionary storing the hourly, daily, and weekly rental rates.
    """
    def __init__(self):
        self.cars = {}
        self.customers = {}
        self.rental_records = {}
        self.rental_rates = {
            'hourly': 15,
            'daily': 100,
            'weekly': 500
        }
        self.next_car_id = 1
        self.next_customer_id = 1

    def add_car(self, make, model, year):
        """Creates and adds a new car to the rental system's inventory."""
        new_car = Car(self.next_car_id, make, model, year)
        self.cars[self.next_car_id] = new_car
        self.next_car_id += 1
        return new_car

    def add_customer(self, name):
        """Creates and adds a new customer to the rental system."""
        new_customer = Customer(self.next_customer_id, name)
        self.customers[self.next_customer_id] = new_customer
        self.next_customer_id += 1
        return new_customer

    def display_available_cars(self):
        """Returns a list of available cars or an empty list if none are available."""
        return [car for car in self.cars.values() if car.is_available]

    def display_customer_rentals(self, customer_id):
        """Returns a list of cars currently rented by a specific customer, including rental basis."""
        customer = self.customers.get(customer_id)
        if not customer or not customer.rented_cars:
            return []

        rentals_list = []
        for car_id, car in customer.rented_cars.items():
            basis = self.rental_records[customer_id][car_id]['basis']
            rentals_list.append((car, basis))
        return rentals_list


    def rent_cars(self, customer_id, requested_rentals):
        """
        Rents one or more cars to a customer.

        Args:
            customer_id (int): The ID of the customer.
            requested_rentals (list): A list of tuples, where each tuple contains
                                      (car_id, rental_basis).

        Returns:
            tuple: (success (bool), message (str))
        """
        if customer_id not in self.customers:
            return False, f"Error: Customer with ID {customer_id} not found."

        cars_to_rent = []

        # Validation Loop
        for car_id, rental_basis in requested_rentals:
            # Check 1: Car exists
            if car_id not in self.cars:
                return False, f"Error: Car with ID {car_id} not found. Rental aborted."

            car = self.cars[car_id]

            # Check 2: Car is available
            if not car.is_available:
                return False, f"Error: Car {car.make} {car.model} (ID: {car_id}) is not available. Rental aborted."

            # Check 3: Rental basis is valid
            if rental_basis not in self.rental_rates:
                return False, f"Error: Invalid rental basis '{rental_basis}'. Please choose from: {list(self.rental_rates.keys())}. Rental aborted."

            cars_to_rent.append((car, rental_basis))

        customer = self.customers[customer_id]

        # Processing Loop
        success_messages = [f"Confirmed Rental Request for {customer.name}:"]
        for car, rental_basis in cars_to_rent:
            car.is_available = False
            customer.rented_cars[car.car_id] = car

            if customer_id not in self.rental_records:
                self.rental_records[customer_id] = {}

            self.rental_records[customer_id][car.car_id] = {
                'start_time': datetime.datetime.now(),
                'basis': rental_basis
            }
            success_messages.append(f"- Rented {car.make} {car.model} (ID: {car.car_id}) on a {rental_basis} basis.")

        return True, "\n".join(success_messages)

    def return_car(self, customer_id, car_id):
        """
        Returns a specific car previously rented by a customer and calculates the bill.

        Returns:
            tuple: (success (bool), message (str) or bill_details (dict))
        """
        if customer_id not in self.customers:
            return False, f"Error: Customer with ID {customer_id} not found."

        customer = self.customers[customer_id]
        if car_id not in customer.rented_cars:
            return False, f"Error: {customer.name} has not rented the car with ID {car_id}."

        car = customer.rented_cars[car_id]
        rental_details = self.rental_records[customer_id].pop(car_id, None)

        if not rental_details:
            return False, f"Error: No rental record found for car ID {car_id}."

        rental_start_time = rental_details['start_time']
        rental_basis = rental_details['basis']

        # Calculate rental duration and cost
        rental_end_time = datetime.datetime.now()
        duration = rental_end_time - rental_start_time

        total_cost = 0.0

        # Calculate cost based on duration and basis
        if rental_basis == 'hourly':
            # Bill per hour (even if partial hour)
            hours = duration.total_seconds() / 3600
            total_cost = self.rental_rates['hourly'] * hours
        elif rental_basis == 'daily':
            # Bill per 24-hour period (rounded up to the next full day)
            days = duration.days + (1 if duration.seconds > 0 else 0)
            total_cost = self.rental_rates['daily'] * days
        elif rental_basis == 'weekly':
            # Bill per 7-day period (rounded up to the next full week)
            weeks = duration.days // 7 + (1 if duration.days % 7 > 0 else 0)
            total_cost = self.rental_rates['weekly'] * weeks

        # Reset car and customer status
        car.is_available = True
        del customer.rented_cars[car_id]

        bill_details = {
            'customer_name': customer.name,
            'car': str(car),
            'rental_basis': rental_basis,
            'start_time': rental_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            'end_time': rental_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            'duration': str(duration),
            'cost': total_cost
        }

        return True, bill_details